In [ ]:
# import statements
from time import sleep
from kafka import KafkaProducer
from json import dumps
import random
import datetime as dt


def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x:dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'climate_data'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    # load in climate_streaming.csv and store in climate
    with open("./climate_streaming.csv") as file:
        climate = file.readlines()
        # strip the new line for each line, and split the attributes into a list
        climate = [line.strip().split(",") for line in climate]
    
    # store the column headers in a variable
    headers = climate[0]
    
    # keep producing records till manual intervention
    while True: 
        # instantiate an empty dict to store result
        data={}
        # randomly pick a row/record in climate data
        temp_data = climate[random.randrange(1,len(climate)-1)]
        # add key-value into data for the chosen record
        for each in headers:
            data[each] = temp_data[headers.index(each)]
        # append sender_id as 1 into data
        data['sender_id'] = 1
        # append created_time into data with the current time
        data['created_time'] = dt.datetime.now().strftime("%X")
        publish_message(producer, topic, 'climate', data)
        # feed a new record every 5 seconds
        sleep(5)